In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.metrics import confusion_matrix

Predict if a user will be an influencer on a tweet (> 100 retweets) based on volume in the following categories

USER PREDICTION ELEMENTS

FIELD                             DESCRIPTION
tweet_user_statuses_count:        count - user tweets
tweet_user_favourites_count:      count - user is a favorite of others
tweet_user_followers_count:       count - user is followed by others
tweet_user_friends_count:         count - user friends
tweet_user_listed_count:          count - user added to other's interest list


In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [3]:
# import necessary libraries
import os
from flask import Flask, render_template, jsonify, request, redirect

from pprint import pprint

#################################################
# Flask Setup
#################################################
app = Flask(__name__)

#################################################
# Database Setup
#################################################

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.sql.expression import func, and_

#Probably don't need these from SQLAlchemy: asc, desc, between, distinct, func, null, nullsfirst, nullslast, or_, and_, not_

db_path_flask_app = "sqlite:///../python/data/twitter_trends.db"
app.config['SQLALCHEMY_DATABASE_URI'] = os.environ.get('DATABASE_URL', '') or db_path_flask_app
# app.config['SQLALCHEMY_DATABASE_URI'] = db_path_flask_app

In [4]:
# Flask-SQLAlchemy database
db = SQLAlchemy(app)

C:\Users\Jeff\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [5]:
class Tweet(db.Model):    
    __tablename__ = 'tweets'    
    # Defining the columns for the table 'tweets',    
    # which will hold tweets associated the search terms in the 'trends' table,    
    # which are referred to in that table as "twitter_tweet_name" 
    
    id = db.Column(db.Integer, primary_key=True)    
    updated_at = db.Column( db.DateTime )    
    tweet_id = db.Column( db.Integer )    
    tweet_id_str = db.Column( db.String(50), unique=True, nullable=False ) 
    
    # tweet_search_term = db.Column(db.Integer, db.ForeignKey('trends.twitter_tweet_name') )    
    tweet_search_term = db.Column(db.String(250))    
    tweet_created_at = db.Column(db.String(100))    
    tweet_is_a_quote_flag = db.Column( db.Boolean )    
    tweet_is_a_retweet_flag = db.Column( db.Boolean )    
    tweet_entities_hashtags_count = db.Column( db.Integer )    
    tweet_entities_user_mentions_count = db.Column( db.Integer )    
    tweet_favorite_counts = db.Column( db.Integer )    
    tweet_retweet_counts = db.Column( db.Integer )    
    tweet_lang = db.Column( db.String(10) )    
    tweet_source = db.Column(db.String(250))    
    tweet_text = db.Column(db.String(250))    
    tweet_user_id = db.Column( db.Integer )    
    tweet_user_id_str = db.Column( db.String(50) )    
    tweet_user_created_at = db.Column(db.String(100))    
    tweet_user_lang = db.Column( db.String(10) )    
    tweet_user_name = db.Column( db.String(100) )    
    tweet_user_screen_name = db.Column( db.String(100) )    
    tweet_user_description = db.Column( db.String(250) )    
    tweet_user_statuses_count = db.Column( db.Integer )    
    tweet_user_favourites_count = db.Column( db.Integer )    
    tweet_user_followers_count = db.Column( db.Integer )    
    tweet_user_friends_count = db.Column( db.Integer )    
    tweet_user_listed_count = db.Column( db.Integer )    
    
    def __repr__(self):        
        return f"<Tweet {self.tweet_search_term}: {self.tweet_id} [updated_at: {self.updated_at}>"

In [6]:
# Database schema for Twitter 'locations' table
class Location(db.Model):
    __tablename__ = 'locations'
    
    # Defining the columns for the table 'locations',
    # which will hold all of the locations in the U.S. for which
    # top trends data is available, as well as location specific
    # info like latitude/longitude
    id = db.Column(db.Integer, primary_key=True)
    updated_at = db.Column( db.DateTime )
    woeid = db.Column(db.Integer, unique=True, nullable=False)
    twitter_country = db.Column(db.String(100))
    tritter_country_code = db.Column(db.String(10))
    twitter_name = db.Column(db.String(250))
    twitter_parentid = db.Column(db.Integer)
    twitter_type = db.Column(db.String(50))
    country_name = db.Column(db.String(250))
    country_name_only = db.Column(db.String(250))
    country_woeid = db.Column(db.Integer)
    county_name = db.Column(db.String(250))
    county_name_only = db.Column(db.String(250))
    county_woeid = db.Column(db.Integer)
    latitude = db.Column(db.Float)
    longitude = db.Column(db.Float)
    name_full = db.Column(db.String(250))
    name_only = db.Column(db.String(250))
    name_woe = db.Column(db.String(250))
    place_type = db.Column(db.String(250))
    state_name = db.Column(db.String(250))
    state_name_only = db.Column(db.String(250))
    state_woeid = db.Column(db.Integer)
    timezone = db.Column(db.String(250))

    my_trends = db.relationship('Trend', backref=db.backref('my_location', lazy=True))
    
    def __repr__(self):
        return f"<Location {self.name_full} [updated_at: {self.updated_at}>"

# Database schema for Twitter 'trends' table
class Trend(db.Model):
    __tablename__ = 'trends'
    
    # Defining the columns for the table 'trends',
    # which will hold all of the top trends associated with
    # locations in the 'locations' table
    id = db.Column(db.Integer, primary_key=True)
    updated_at = db.Column( db.DateTime )
    woeid = db.Column(db.Integer, db.ForeignKey('locations.woeid') )
    twitter_as_of = db.Column(db.String(100))
    twitter_created_at = db.Column(db.String(100))
    twitter_name = db.Column(db.String(250))
    twitter_tweet_name = db.Column(db.String(250))
    twitter_tweet_promoted_content = db.Column(db.String(250))
    twitter_tweet_query = db.Column(db.String(250))
    twitter_tweet_url = db.Column(db.String(250))
    twitter_tweet_volume = db.Column(db.Float)

    # With more investigation, determined this is an
    # incorrect usage of relationship method below - removing it
    # locations = db.relationship('Location', backref=db.backref('trends', lazy=True))
     
    def __repr__(self):
        return f"<Trend {self.my_location.name_full}: {self.twitter_tweet_name} [updated_at: {self.updated_at}>"


In [7]:
# In the app code for this route, there is no need to do anything special
a_tweet = "#OpeningDay"

# Return a list of all locations that have the specified tweet in its top trends
# and then sort the results by tweet volume in descending order
results = db.session.query(Trend, Location).join(Location) \
                    .filter(Trend.twitter_tweet_name == a_tweet ) \
                    .order_by( Trend.twitter_tweet_volume.desc() ).all()

loc_list = []
for r in results:
    #print(f"Trend Information for {r.Trend.woeid} {r.Location.name_full}: {r.Trend.twitter_tweet_name} {r.Trend.twitter_tweet_volume}")
    loc_info = {
        'woeid': r.Location.woeid,
        'latitude': r.Location.latitude,
        'longitude': r.Location.longitude,
        'name_full': r.Location.name_full,
        'name_only': r.Location.name_only,
        'name_woe': r.Location.name_woe,
        'county_name': r.Location.county_name,
        'county_name_only': r.Location.county_name_only,
        'county_woeid': r.Location.county_woeid,
        'state_name': r.Location.state_name,
        'state_name_only': r.Location.state_name_only,
        'state_woeid': r.Location.state_woeid,
        'country_name': r.Location.country_name,
        'country_name_only': r.Location.country_name_only,
        'country_woeid': r.Location.country_woeid,
        'place_type': r.Location.place_type,
        'timezone': r.Location.timezone,
        'twitter_type': r.Location.twitter_type,
        'twitter_country': r.Location.twitter_country,
        'tritter_country_code': r.Location.tritter_country_code,
        'twitter_parentid': r.Location.twitter_parentid,

        'twitter_as_of': r.Trend.twitter_as_of,
        'twitter_created_at': r.Trend.twitter_created_at,
        'twitter_name': r.Trend.twitter_name,
        'twitter_tweet_name': r.Trend.twitter_tweet_name,
        'twitter_tweet_promoted_content': r.Trend.twitter_tweet_promoted_content,
        'twitter_tweet_query': r.Trend.twitter_tweet_query,
        'twitter_tweet_url': r.Trend.twitter_tweet_url,
        'twitter_tweet_volume': r.Trend.twitter_tweet_volume
    }

    loc_list.append(loc_info)

In [8]:
results = db.session.query(Tweet.tweet_user_id_str, \
                        func.max(Tweet.id).label("id"), \
                        func.max(Tweet.tweet_user_id_str).label("tweet_user_id_str"), \
                        func.max(Tweet.tweet_user_statuses_count).label("tweet_user_statuses_count"), \
                        func.max(Tweet.tweet_user_favourites_count).label("tweet_user_favourites_count"), \
                        func.max(Tweet.tweet_user_followers_count).label("tweet_user_followers_count"), \
                        func.max(Tweet.tweet_user_friends_count).label("tweet_user_friends_count"), \
                        func.max(Tweet.tweet_user_listed_count).label("tweet_user_listed_count"), \
                        func.max(Tweet.tweet_retweet_counts).label("tweet_retweet_counts")) \
                        .group_by(Tweet.tweet_user_id_str) \
                        .all()        

tweet_list = []    

for r in results:        
    tweet_info = { 'id': r.id,            
                  'tweet_user_id_str' : r.tweet_user_id_str ,                                           
                  'tweet_user_statuses_count': r.tweet_user_statuses_count ,            
                  'tweet_user_favourites_count' : r.tweet_user_favourites_count ,            
                  'tweet_user_followers_count' : r.tweet_user_followers_count ,            
                  'tweet_user_friends_count' : r.tweet_user_friends_count ,            
                  'tweet_user_listed_count' : r.tweet_user_listed_count,    
                  'tweet_retweet_counts' : r.tweet_retweet_counts}        
    tweet_list.append(tweet_info)

In [9]:
pprint(len(tweet_list))

32049


In [10]:
tweet = pd.DataFrame.from_dict(tweet_list)
tweet.head()

,id,tweet_retweet_counts,tweet_user_favourites_count,tweet_user_followers_count,tweet_user_friends_count,tweet_user_id_str,tweet_user_listed_count,tweet_user_statuses_count
0,26060,91,773,250,196,1000070228263493633,1,870
1,20218,31,5091,294,333,1000075196827566080,0,3260
2,34658,51,2154,65,128,1000088820652675072,0,279
3,33000,0,19169,532,286,1000090083821785088,2,3491
4,24629,1389,1084,41,143,1000128848455847936,0,3937


In [11]:
tweet.describe()

,id,tweet_retweet_counts,tweet_user_favourites_count,tweet_user_followers_count,tweet_user_friends_count,tweet_user_listed_count,tweet_user_statuses_count
count,32049.000000,32049.000000,3.204900e+04,3.204900e+04,3.204900e+04,32049.000000,3.204900e+04
mean,19912.455989,1294.090393,1.624379e+04,1.080645e+04,1.575968e+03,95.979188,2.562619e+04
std,11287.823288,11921.900995,3.745311e+04,3.269520e+05,8.848977e+03,1225.066403,6.977079e+04
min,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000e+00
25%,10214.000000,0.000000,6.480000e+02,1.200000e+02,1.850000e+02,0.000000,1.357000e+03
50%,20099.000000,3.000000,3.837000e+03,4.070000e+02,4.720000e+02,4.000000,6.246000e+03
75%,29682.000000,73.000000,1.496200e+04,1.417000e+03,1.215000e+03,28.000000,2.299000e+04
max,39031.000000,533302.000000,1.005418e+06,4.186532e+07,1.057259e+06,138404.000000,3.097116e+06


In [12]:
tweet["success"] = np.where(tweet["tweet_retweet_counts"] > 99, 1, 0)

In [13]:
tweet.head()

,id,tweet_retweet_counts,tweet_user_favourites_count,tweet_user_followers_count,tweet_user_friends_count,tweet_user_id_str,tweet_user_listed_count,tweet_user_statuses_count,success
0,26060,91,773,250,196,1000070228263493633,1,870,0
1,20218,31,5091,294,333,1000075196827566080,0,3260,0
2,34658,51,2154,65,128,1000088820652675072,0,279,0
3,33000,0,19169,532,286,1000090083821785088,2,3491,0
4,24629,1389,1084,41,143,1000128848455847936,0,3937,1


In [14]:
# Assign X (data) and y (target)
X = tweet.drop(["id","tweet_user_id_str","tweet_retweet_counts","success"], axis=1)
y = tweet["success"]
print(X.shape, y.shape)

(32049, 5) (32049,)


In [15]:
X.head(5)

,tweet_user_favourites_count,tweet_user_followers_count,tweet_user_friends_count,tweet_user_listed_count,tweet_user_statuses_count
0,773,250,196,1,870
1,5091,294,333,0,3260
2,2154,65,128,0,279
3,19169,532,286,2,3491
4,1084,41,143,0,3937


Split our data into training and testing

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

Scale the training and testing data

In [17]:
from sklearn.preprocessing import StandardScaler
# Create a StandardScater model and fit it to the training data
# No need to scale y since it is just 0 and 1
X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)

In [18]:
# Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

Create a Logistic Regression Model

In [19]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Fit (train) or model using the training data

In [20]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Validate the model using the test data

In [21]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.7653103677816608
Testing Data Score: 0.7660052414825907


In [22]:
# RESULTS OF RUN - No Scaling
# Training Data Score: 0.7661008487269096
# Testing Data Score: 0.7675028079371022

# RESULTS OF RUN - With Scaling
# Training Data Score: 0.7653103677816608
# Testing Data Score: 0.7660052414825907

Make predictions

In [23]:
predictions = classifier.predict(X_test_scaled)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0 0 0 0 0 0 0 0 0 0]
First 10 Actual labels: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [24]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,0,0
1,0,0
2,0,1
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [25]:
confusion_matrix(y_test, predictions)

array([[6105,   41],
       [1834,   33]], dtype=int64)

In [26]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn: 6105   fp:  41
fn: 1834   tp:  33


In [27]:
# from joblib import dump, load
# dump(classifier, 'influencer.model') 

# Functionalize the prediction code

In [28]:
# Function to load the 'influencer.model' from a specified file
from joblib import dump, load

def influencer_load_model():
    # influencer_model_file = "resources/influencer.model"
    influencer_model_file = "influencer.model"
    
    classifer_model = load(influencer_model_file)
    
    return classifer_model 

In [29]:
def get_user_info(a_tweet_user_id_str=None):
# Function to find the most recent entry in the tweets table for the specified user
# Argument: tweet_user_id_str
#
# Returns: Dictionary of (tweet and) user information

    # Check for null user id string
    if a_tweet_user_id_str is None:
        return None

    
    # Create a subquery to find the most recent "updated_at" record per tweet_user_id_str
    user_subq = db.session.query(Tweet.tweet_user_id_str, func.max(Tweet.updated_at).label("max_updated_at")) \
                        .group_by(Tweet.tweet_user_id_str).subquery()
    
    # Find the most recent entry in the Tweets table for this user
    r = db.session.query(Tweet) \
                        .filter( and_(
                            Tweet.tweet_user_id_str == a_tweet_user_id_str, \
                            Tweet.tweet_user_id_str == user_subq.c.tweet_user_id_str, \
                            Tweet.updated_at == user_subq.c.max_updated_at \
                        )) \
                        .first()
        
    # Populate the results in a dictionary
    tweet_info = {
        'id': r.id,
        'updated_at' : r.updated_at,

        'tweet_id' : r.tweet_id ,
        'tweet_id_str' : r.tweet_id_str,
        'tweet_search_term' : r.tweet_search_term,
        'tweet_created_at' : r.tweet_created_at,

        'tweet_is_a_quote_flag' : r.tweet_is_a_quote_flag,
        'tweet_is_a_retweet_flag' : r.tweet_is_a_retweet_flag,

        'tweet_entities_hashtags_count' : r.tweet_entities_hashtags_count,
        'tweet_entities_user_mentions_count' : r.tweet_entities_user_mentions_count,
        'tweet_favorite_counts' : r.tweet_favorite_counts,
        'tweet_retweet_counts' : r.tweet_retweet_counts,

        'tweet_lang' : r.tweet_lang,
        'tweet_source': r.tweet_source,
        'tweet_text' : r.tweet_text,

        'tweet_user_id' : r.tweet_user_id,
        'tweet_user_id_str' : r.tweet_user_id_str ,
        'tweet_user_created_at' : r.tweet_user_created_at,
        'tweet_user_lang' : r.tweet_user_lang ,
        'tweet_user_name' : r.tweet_user_name ,
        'tweet_user_screen_name' : r.tweet_user_screen_name ,
        'tweet_user_description' : r.tweet_user_description ,
        'tweet_user_statuses_count': r.tweet_user_statuses_count ,
        'tweet_user_favourites_count' : r.tweet_user_favourites_count ,
        'tweet_user_followers_count' : r.tweet_user_followers_count ,
        'tweet_user_friends_count' : r.tweet_user_friends_count ,
        'tweet_user_listed_count' : r.tweet_user_listed_count 
    }
    
    # Return a dictionary of tweet and user information for this user
    return tweet_info

In [30]:
user_info = get_user_info("1000075196827566080")
pprint(user_info)

{'id': 20213,
 'tweet_created_at': 'Mon May 06 11:47:36 +0000 2019',
 'tweet_entities_hashtags_count': 4,
 'tweet_entities_user_mentions_count': 0,
 'tweet_favorite_counts': 4,
 'tweet_id': 1125366501136113664,
 'tweet_id_str': '1125366501136113664',
 'tweet_is_a_quote_flag': False,
 'tweet_is_a_retweet_flag': False,
 'tweet_lang': 'en',
 'tweet_retweet_counts': 2,
 'tweet_search_term': 'TeamDCS',
 'tweet_source': '<a href="http://twitter.com/download/iphone" '
                 'rel="nofollow">Twitter for iPhone</a>',
 'tweet_text': 'Happy Teacher Appreciation Week, #TeamDCS  We appreciate all '
               'you do!  #RaisetheBar #RTBChat #allmeansall '
               'https://t.co/Z1u83WUXGW',
 'tweet_user_created_at': 'Fri May 25 18:04:42 +0000 2018',
 'tweet_user_description': 'All things elementary for DeSoto County Schools '
                           '#TeamDCS #RaisetheBar',
 'tweet_user_favourites_count': 5091,
 'tweet_user_followers_count': 294,
 'tweet_user_friends_count': 

In [31]:
def predict_influencer(a_tweet_user_id_str, a_classifier):
# Function to provide a prediction of a specified being an "Influencer"
# "Influencer" = A user who is likely to have >99 retweets from other Twitter users
#
# Argument: tweet_user_id_str
#
# Returns:
#     True: User is predicted to be an Influencer
#     False: User is predicted to not be an Influencer

    # Check for null user id string
    if a_tweet_user_id_str is None:
        return None

    # Ensure the influencer model global varable is populated
    if a_classifier == None:
        return None
    
    # Get the most recent info for this user from the tweets table
    user_info = get_user_info(a_tweet_user_id_str)
    
    # Setup the info needed to get a prediction
    # Key = columns, Value = list of row values for each column
    x_info = {
        'tweet_user_favourites_count': [user_info['tweet_user_favourites_count']],
        'tweet_user_followers_count': [user_info['tweet_user_followers_count']],
        'tweet_user_friends_count': [user_info['tweet_user_friends_count']],
        'tweet_user_listed_count': [user_info['tweet_user_listed_count']],
        'tweet_user_statuses_count': [user_info['tweet_user_statuses_count']]
    }
    
    # Populate the info needed for prediction into a dataframe
    X = pd.DataFrame.from_dict( x_info )
    
    # Make a prediction
    y_predicted = a_classifier.predict(X)
    
    # Convert the prediction to True (Influencer) / False (Not Influencer) value
    influencer_predicted = True if y_predicted > 0 else False
    
    results = {
        'influencer_predicted': influencer_predicted,
        'y_predicted': y_predicted[0],
#        'tweet_retweet_counts': user_info['tweet_retweet_counts'],
#        'influencer_actual': user_info['tweet_retweet_counts'] > 99,
        'x_info': x_info,
        'user_info': user_info
    }
    
    return results

In [32]:
# Load the influencer ML model
influencer_classifier = influencer_load_model()

In [34]:
user_influencer_prediction = predict_influencer('1000075196827566080', influencer_classifier)
pprint(user_influencer_prediction)

{'influencer_predicted': False,
 'user_info': {'id': 20213,
               'tweet_created_at': 'Mon May 06 11:47:36 +0000 2019',
               'tweet_entities_hashtags_count': 4,
               'tweet_entities_user_mentions_count': 0,
               'tweet_favorite_counts': 4,
               'tweet_id': 1125366501136113664,
               'tweet_id_str': '1125366501136113664',
               'tweet_is_a_quote_flag': False,
               'tweet_is_a_retweet_flag': False,
               'tweet_lang': 'en',
               'tweet_retweet_counts': 2,
               'tweet_search_term': 'TeamDCS',
               'tweet_source': '<a href="http://twitter.com/download/iphone" '
                               'rel="nofollow">Twitter for iPhone</a>',
               'tweet_text': 'Happy Teacher Appreciation Week, #TeamDCS  We '
                             'appreciate all you do!  #RaisetheBar #RTBChat '
                             '#allmeansall https://t.co/Z1u83WUXGW',
               'tweet_u

In [35]:
def get_user_tweet_list(a_tweet_user_id_str=None):
# Function to find the list of all tweets in the tweets table sent by the specified user
# Argument: tweet_user_id_str
#
# Returns: List of Dictionaries of (tweet and) user information

    # Check for null user id string
    if a_tweet_user_id_str is None:
        return None

    # Find the most recent entry in the Tweets table for this user
    results = db.session.query(Tweet) \
                        .filter(Tweet.tweet_user_id_str == a_tweet_user_id_str) \
                        .order_by( Tweet.tweet_created_at ) \
                        .all()

    tweet_list = []
    
    for r in results:
        # Populate the results in a dictionary
        tweet_info = {
            'id': r.id,
            'updated_at' : r.updated_at,

            'tweet_id' : r.tweet_id ,
            'tweet_id_str' : r.tweet_id_str,
            'tweet_search_term' : r.tweet_search_term,
            'tweet_created_at' : r.tweet_created_at,

            'tweet_is_a_quote_flag' : r.tweet_is_a_quote_flag,
            'tweet_is_a_retweet_flag' : r.tweet_is_a_retweet_flag,

            'tweet_entities_hashtags_count' : r.tweet_entities_hashtags_count,
            'tweet_entities_user_mentions_count' : r.tweet_entities_user_mentions_count,
            'tweet_favorite_counts' : r.tweet_favorite_counts,
            'tweet_retweet_counts' : r.tweet_retweet_counts,

            'tweet_lang' : r.tweet_lang,
            'tweet_source': r.tweet_source,
            'tweet_text' : r.tweet_text,

            'tweet_user_id' : r.tweet_user_id,
            'tweet_user_id_str' : r.tweet_user_id_str ,
            'tweet_user_created_at' : r.tweet_user_created_at,
            'tweet_user_lang' : r.tweet_user_lang ,
            'tweet_user_name' : r.tweet_user_name ,
            'tweet_user_screen_name' : r.tweet_user_screen_name ,
            'tweet_user_description' : r.tweet_user_description ,
            'tweet_user_statuses_count': r.tweet_user_statuses_count ,
            'tweet_user_favourites_count' : r.tweet_user_favourites_count ,
            'tweet_user_followers_count' : r.tweet_user_followers_count ,
            'tweet_user_friends_count' : r.tweet_user_friends_count ,
            'tweet_user_listed_count' : r.tweet_user_listed_count 
        }
        
        # Append it to the list
        tweet_list.append(tweet_info)
        
    # Return the results
    return tweet_list

In [36]:
user_tweet_list = get_user_tweet_list( '1000075196827566080' )
print(len (user_tweet_list) )
pprint(user_tweet_list)

6
[{'id': 20218,
  'tweet_created_at': 'Mon May 06 11:45:22 +0000 2019',
  'tweet_entities_hashtags_count': 0,
  'tweet_entities_user_mentions_count': 1,
  'tweet_favorite_counts': 0,
  'tweet_id': 1125365937794891776,
  'tweet_id_str': '1125365937794891776',
  'tweet_is_a_quote_flag': False,
  'tweet_is_a_retweet_flag': True,
  'tweet_lang': 'en',
  'tweet_retweet_counts': 31,
  'tweet_search_term': 'TeamDCS',
  'tweet_source': '<a href="http://twitter.com/download/iphone" '
                  'rel="nofollow">Twitter for iPhone</a>',
  'tweet_text': 'RT @cory_uselton: This is Teacher Appreciation Week. We are '
                'thankful for our awesome teachers &amp; teacher assistants, '
                'and we hope that everyo…',
  'tweet_user_created_at': 'Fri May 25 18:04:42 +0000 2018',
  'tweet_user_description': 'All things elementary for DeSoto County Schools '
                            '#TeamDCS #RaisetheBar',
  'tweet_user_favourites_count': 5091,
  'tweet_user_followers_co

In [37]:
# Create a list of retweet counts for this user
retweet_counts = [ x['tweet_retweet_counts'] for x in user_tweet_list ]
retweet_counts

[31, 3, 4, 4, 4, 2]

In [38]:
np.arange(len(retweet_counts))

array([0, 1, 2, 3, 4, 5])

In [42]:
from api_config import (plotly_python_username, plotly_python_api_key)


import plotly
plotly.tools.set_credentials_file(username=plotly_python_username, api_key=plotly_python_api_key)

import plotly.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np
# Create a trace
# trace = go.Scatter(
#     x = np.arange(len( retweet_counts) ),
#     y = retweet_counts,
#     mode = 'markers'
# )

# Create a trace
trace = go.Bar(
    x = np.arange(len( retweet_counts) ),
    y = retweet_counts
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data, filename='retweet_chart')

# or plot with: plot_url = py.plot(data, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~daddyjab/0 or inside your plot.ly account where it is named 'retweet_chart'
